In [ ]:
import os, sys
sys.path.append(os.path.abspath(".."))

import torch
from nanochat.gpt import GPT, GPTConfig

# 1. 定义配置 (这里使用一个小模型作为演示)
config = GPTConfig(
    vocab_size=50304, # 词表大小
    sequence_len=1024,# 最大上下文长度
    n_layer=4,        # Transformer 层数
    n_head=4,         # 注意力头数
    n_kv_head=4,   
    n_embd=128        # 嵌入维度
)

# 2. 实例化模型
model = GPT(config)

# 3. 初始化权重 (关键步骤)
# 应用特定的正态分布初始化，并将 lm_head 和部分投影层置零
model.init_weights()

print(f"模型参数量: {sum(p.numel() for p in model.parameters())}")

# --- 场景 A: 推理 (Inference) ---
# 构造模拟输入: Batch Size=1, Sequence Length=5
B, T = 1, 5
idx = torch.randint(0, config.vocab_size, (B, T))

# 前向传播 (不传 targets)
logits = model(idx)

# 输出形状: (Batch, Seq_Len, Vocab_Size)
print(f"\n输入形状: {idx.shape}")
print(f"Logits 形状: {logits.shape}") # -> torch.Size([2, 16, 50304])


# --- 场景 B: 训练 (Training) ---
# 构造模拟目标 (通常是 idx 向后移一位)
targets = torch.randint(0, config.vocab_size, (B, T))

# 前向传播 (传入 targets)
loss = model(idx, targets=targets)

# 输出是一个标量 Loss
print(f"\nTraining Loss: {loss.item():.4f}")

模型参数量: 13664256

输入形状: torch.Size([1, 5])
Logits 形状: torch.Size([1, 5, 50304])

Training Loss: 10.8258


In [4]:
idx

tensor([[25755,  1124, 38056, 37306, 17460]])

In [26]:
logits

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], grad_fn=<MulBackward0>)

In [23]:
idx

tensor([[37676,  4953, 38569, 32074,  1710]])

In [24]:
next_token = logits[:, -1, :]
print(f"next_token形状: {next_token.shape}")

next_token形状: torch.Size([1, 50304])


In [1]:
import torch
import torch.nn as nn

# --- 1. 定义线性层 ---
# 输入特征数=2 (身高, 体重)
# 输出特征数=3 (健康指数, 体脂率, 肌肉量)
# bias=True (默认开启，允许加一个基础偏移值)
layer = nn.Linear(in_features=2, out_features=3, bias=True)

# 查看随机初始化的权重矩阵 W (形状是 3x2)
print("权重矩阵 W (3x2):")
print(layer.weight)
print("\n偏置向量 b (3):")
print(layer.bias)

# --- 2. 准备数据 ---
# 模拟 2 个人：
# A: 身高 1.8, 体重 80
# B: 身高 1.6, 体重 50
# 输入形状: (Batch_Size=2, Input_Features=2)
x = torch.tensor([
    [1.8, 80.0], 
    [1.6, 50.0]
])

# --- 3. 前向传播 ---
# 自动执行 y = x @ W.T + b
y = layer(x)

print("\n--- 计算结果 ---")
print(f"输入形状: {x.shape}  # [2, 2]")
print(f"输出形状: {y.shape}  # [2, 3] -> 维度变了！")
print("\n输出值 (模拟的[健康指数, 体脂率, 肌肉量]):")
print(y)

# --- 4. 手动验证 (揭示本质) ---
# y = x * W_transposed + b
print("\n--- 手动验证 (数学本质) ---")
manual_y = torch.matmul(x, layer.weight.t()) + layer.bias
print("手动计算结果与自动计算一致:", torch.allclose(y, manual_y))

权重矩阵 W (3x2):
Parameter containing:
tensor([[-0.1126, -0.3855],
        [ 0.0426,  0.5514],
        [-0.3952,  0.6278]], requires_grad=True)

偏置向量 b (3):
Parameter containing:
tensor([ 0.1770, -0.0633,  0.4804], requires_grad=True)

--- 计算结果 ---
输入形状: torch.Size([2, 2])  # [2, 2]
输出形状: torch.Size([2, 3])  # [2, 3] -> 维度变了！

输出值 (模拟的[健康指数, 体脂率, 肌肉量]):
tensor([[-30.8697,  44.1222,  49.9896],
        [-19.2807,  27.5728,  31.2360]], grad_fn=<AddmmBackward0>)

--- 手动验证 (数学本质) ---
手动计算结果与自动计算一致: True


In [2]:
layer.weight

Parameter containing:
tensor([[-0.1126, -0.3855],
        [ 0.0426,  0.5514],
        [-0.3952,  0.6278]], requires_grad=True)